# flashgeotext

Get country data (+synonyms)

In [ ]:
import pandas as pd
from flashtext import KeywordProcessor
import io
from fuzzywuzzy import fuzz
import requests
import pycountry
import bs4
import requests
import json

In [ ]:
url = "https://en.wikipedia.org/wiki/List_of_alternative_country_names"

response = requests.get(url)

soup = bs4.BeautifulSoup(response.text, "html5lib")

In [ ]:
table_bodies = (table_body for table_body in soup.find_all('tbody'))

In [ ]:
data = {}
for table_body in table_bodies:
    rows = table_body.find_all('tr')
    for row in rows:
        cols = row.find_all('td')
        if cols:
            country_name = cols[0].find("a").text
            data[country_name] = [b.text for b in cols[1].find_all("b")] + [country_name]
            data[country_name] = [word for word in data[country_name] if fuzz.ratio(country_name, word) > 20]

# weird geopolitical overtaking of geonames.org
data["Taiwan"] = [word for word in data["Taiwan"] if "China" not in word]

In [ ]:
with open("countries.json", "w", encoding="utf8") as file:
    json.dump(data, file, ensure_ascii=False)